In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.stats import rankdata
import pickle
import statistics as stat
import math

/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


Load each lineage - either low or high clustering resolution

In [2]:
endo = sc.read_h5ad("../data/endo.dpt.h5ad")
hep = sc.read_h5ad("../data/hep.dpt.h5ad")
neur = sc.read_h5ad("../data/neur.dpt.h5ad")

In [2]:
endo = sc.read_h5ad("../data/endo.dpt.hires.h5ad")
hep = sc.read_h5ad("../data/hep.dpt.hires.h5ad")
neur = sc.read_h5ad("../data/neur.dpt.hires.h5ad")

Useful functions for pseudobulk aggregation

In [3]:
def aggregate(sc_sub):
    """
    input: sc_sub, a scanpy h5ad object, [cells x genes]
    output: mat, a csr matrix, [1 x genes], summed over cells
    """
    mat = csr_matrix.sum(sc_sub.X, 0)
    return(mat)

In [4]:
def log_normalize(mat):
    """
    input: mat, a matrix, [1 x genes], containing raw counts
    output: logcp10k, a csr matrix, [1 x genes], = log10(counts per 10k UMI + 1)
    """
    cp10k = mat * 1e4 / np.sum(mat, 1)
    logcp10k = np.log10(cp10k + 1)
    return(logcp10k)

In [5]:
def scale(mat, center=True):
    """
    input: mat, a [levels x gene] csr matrix
           center, boolean indicating whether or not you want to center the data
    output: scaled_mat, a [levels x gene] csr matrix where each gene has zero mean, unit variance across all levels
    """
    if center:
        mat = mat - np.mean(mat, 0)
    stdevs = np.std(mat, 0)
    stdevs[stdevs == 0] = 1
    scaled_mat = mat / stdevs
    return(scaled_mat)

Master function for pseudobulk aggregation

In [6]:
def get_pseudobulk(sc_obj, stratifier="individual", pt_label="dpt_pseudotime", nbins=5):
    """
    input: sc_obj, a full h5ad object
           stratifier, the string identifier for the category over which pseudobulk will be aggregated (along with pseudotime)
           pt_label, the string identifier for pseudotime, ie 'dpt_pseudotime'
           nbins, the number of pseudotime bins to compute
    output: Y, an [NGT, 1] array where N=number of levels to stratifier, G=num genes, T=nbins
            X, an [NGT, 3] matrix (N,G,T as above), where the first column is time, second column is level, third is gene
            leveldict, a dictionary mapping ints in second column of X to unique values in sc_obj.obs[stratifier]
            genedict, a dictionary mapping ints in third column of X to gene names in sc_obj
            cellcounts, 
    """
    levels = np.unique(sc_obj.obs[stratifier])
    bin_width = 1/nbins
    nL = len(levels)
    nG = sc_obj.X.shape[1]
    pseudobulk = np.zeros([nbins*nL, nG])
    X = np.zeros([nbins*nL*nG, 3])
    cellcounts = np.zeros(nbins*nL, dtype=int)
    empty = []
    for t in range(nbins):
        if t < nbins-1:
            t_subset = (t*bin_width <= sc_obj.obs[pt_label]) & (sc_obj.obs[pt_label] < (t+1)*bin_width)
        else:
            t_subset = (t*bin_width <= sc_obj.obs[pt_label]) & (sc_obj.obs[pt_label] <= (t+1)*bin_width)
        for l in range(nL):
            l_subset = sc_obj.obs[stratifier] == levels[l]
            sub = sc_obj[l_subset & t_subset]
            # get cell counts for this subset
            cellcounts[t*nL + l] = sub.X.shape[0]
            # get metadata, store in X
            t_sub = np.median(sub.obs[pt_label])
            X[(t*nL*nG+l*nG):(t*nL*nG+(l+1)*nG),0] = t_sub
            X[(t*nL*nG+l*nG):(t*nL*nG+(l+1)*nG),1] = l
            X[(t*nL*nG+l*nG):(t*nL*nG+(l+1)*nG),2] = range(nG)
            # get expression data, store in pseudobulk
            sub_sum = aggregate(sub)
            sub_cpm = log_normalize(sub_sum)
            pseudobulk[t*nL + l, :] = sub_cpm
            # if there is a sample with zero cells, we'll remember that and delete it 
            if cellcounts[t*nL + l] == 0:
                empty.append(t*nL + l)
    if sum(cellcounts==0) > 0:
        print("Removing " + str(sum(cellcounts==0)) + " empty bins")
    X = X[~np.isnan(X[:,0])]
    pseudobulk = np.delete(pseudobulk, empty, axis=0)
    pseudobulk = scale(pseudobulk)
    Y = pseudobulk.flatten()
    leveldict = {i:levels[i] for i in range(nL)}
    genedict = {sc_obj.var_names[i]:i for i in range(nG)}
    
    return({"Y":Y,"X":X,"level_dict":leveldict, "gene_dict":genedict, "cell_counts":cellcounts})

## Endothelial Lineage

### Stratify by individual

In [7]:
endo_ind = get_pseudobulk(endo, stratifier="individual")

In [196]:
np.savetxt( "../data/endo.dpt.ind.X.txt", endo_ind['X'])
np.savetxt("../data/endo.dpt.ind.Y.txt", endo_ind['Y'])
with open("../data/endo.dpt.ind.genedict.pickle", 'wb') as f:
    pickle.dump(endo_ind['gene_dict'], f)
    f.close()

In [8]:
np.savetxt( "../data/endo.dpt.hires.ind.X.txt", endo_ind['X'])
np.savetxt("../data/endo.dpt.hires.ind.Y.txt", endo_ind['Y'])
with open("../data/endo.dpt.hires.ind.genedict.pickle", 'wb') as f:
    pickle.dump(endo_ind['gene_dict'], f)
    f.close()

### Stratify by batch

In [9]:
endo_batch = get_pseudobulk(endo, stratifier="Batch")

In [113]:
np.savetxt( "../data/endo.dpt.batch.X.txt", endo_batch['X'])
np.savetxt("../data/endo.dpt.batch.Y.txt", endo_batch['Y'])
with open("../data/endo.dpt.batch.genedict.pickle", 'wb') as f:
    pickle.dump(endo_batch['gene_dict'], f)
    f.close()

In [10]:
np.savetxt( "../data/endo.dpt.hires.batch.X.txt", endo_batch['X'])
np.savetxt("../data/endo.dpt.hires.batch.Y.txt", endo_batch['Y'])
with open("../data/endo.dpt.hires.batch.genedict.pickle", 'wb') as f:
    pickle.dump(endo_batch['gene_dict'], f)
    f.close()

### Stratify by both batch and individual

In [11]:
endo.obs['batchind'] = [endo.obs['individual'][i] + '--' + endo.obs['Batch'][i] for i in range(endo.X.shape[0])]

In [12]:
endo_batchind = get_pseudobulk(endo, stratifier="batchind", nbins=4)

/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work-zfs/abattle4/josh/ebpilot/myenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work-zfs/abattle4/josh/ebpilot/myenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide


Removing 2 empty bins


In [11]:
np.savetxt( "../data/endo.dpt.batchind.X.txt", endo_batchind['X'])
np.savetxt("../data/endo.dpt.batchind.Y.txt", endo_batchind['Y'])
with open("../data/endo.dpt.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(endo_batchind['gene_dict'], f)
    f.close()
with open("../data/endo.dpt.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(endo_batchind['level_dict'], f)
    f.close()

In [13]:
np.savetxt( "../data/endo.dpt.hires.batchind.X.txt", endo_batchind['X'])
np.savetxt("../data/endo.dpt.hires.batchind.Y.txt", endo_batchind['Y'])
with open("../data/endo.dpt.hires.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(endo_batchind['gene_dict'], f)
    f.close()
with open("../data/endo.dpt.hires.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(endo_batchind['level_dict'], f)
    f.close()

## Hepatocyte lineage

### Stratified by individual

In [14]:
hep_ind = get_pseudobulk(hep, stratifier="individual", nbins=5)

Removing 1 empty bins


/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work-zfs/abattle4/josh/ebpilot/myenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide


In [199]:
np.savetxt( "../data/hep.dpt.ind.X.txt", hep_ind['X'])
np.savetxt("../data/hep.dpt.ind.Y.txt", hep_ind['Y'])
with open("../data/hep.dpt.ind.genedict.pickle", 'wb') as f:
    pickle.dump(hep_ind['gene_dict'], f)
    f.close()

In [15]:
np.savetxt( "../data/hep.dpt.hires.ind.X.txt", hep_ind['X'])
np.savetxt("../data/hep.dpt.hires.ind.Y.txt", hep_ind['Y'])
with open("../data/hep.dpt.hires.ind.genedict.pickle", 'wb') as f:
    pickle.dump(hep_ind['gene_dict'], f)
    f.close()

### Stratified by batch

In [16]:
hep_batch = get_pseudobulk(hep, stratifier="Batch")

In [118]:
np.savetxt( "../data/hep.dpt.batch.X.txt", hep_batch['X'])
np.savetxt("../data/hep.dpt.batch.Y.txt", hep_batch['Y'])
with open("../data/hep.dpt.batch.genedict.pickle", 'wb') as f:
    pickle.dump(hep_batch['gene_dict'], f)
    f.close()

In [17]:
np.savetxt( "../data/hep.dpt.hires.batch.X.txt", hep_batch['X'])
np.savetxt("../data/hep.dpt.hires.batch.Y.txt", hep_batch['Y'])
with open("../data/hep.dpt.hires.batch.genedict.pickle", 'wb') as f:
    pickle.dump(hep_batch['gene_dict'], f)
    f.close()

### Stratified by batch and individual

In [18]:
hep.obs['batchind'] = [hep.obs['individual'][i] + '--' + hep.obs['Batch'][i] for i in range(hep.X.shape[0])]

In [19]:
hep_batchind = get_pseudobulk(hep, stratifier="batchind", nbins=5)

/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work-zfs/abattle4/josh/ebpilot/myenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home-2/jpopp4@jhu.edu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work-zfs/abattle4/josh/ebpilot/myenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide
/home-2/jpopp4@jhu.edu/.local/lib/python3.

Removing 5 empty bins


In [17]:
np.savetxt( "../data/hep.dpt.batchind.X.txt", hep_batchind['X'])
np.savetxt("../data/hep.dpt.batchind.Y.txt", hep_batchind['Y'])
with open("../data/hep.dpt.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(hep_batchind['gene_dict'], f)
    f.close()
with open("../data/hep.dpt.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(hep_batchind['level_dict'], f)
    f.close()

In [20]:
np.savetxt( "../data/hep.dpt.hires.batchind.X.txt", hep_batchind['X'])
np.savetxt("../data/hep.dpt.hires.batchind.Y.txt", hep_batchind['Y'])
with open("../data/hep.dpt.hires.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(hep_batchind['gene_dict'], f)
    f.close()
with open("../data/hep.dpt.hires.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(hep_batchind['level_dict'], f)
    f.close()

## Neuronal lineage

### Stratified by individual

In [21]:
neur_ind = get_pseudobulk(neur, stratifier="individual")

In [120]:
np.savetxt( "../data/neur.dpt.ind.X.txt", neur_ind['X'])
np.savetxt("../data/neur.dpt.ind.Y.txt", neur_ind['Y'])
with open("../data/neur.dpt.ind.genedict.pickle", 'wb') as f:
    pickle.dump(neur_ind['gene_dict'], f)
    f.close()

In [22]:
np.savetxt( "../data/neur.dpt.hires.ind.X.txt", neur_ind['X'])
np.savetxt("../data/neur.dpt.hires.ind.Y.txt", neur_ind['Y'])
with open("../data/neur.dpt.hires.ind.genedict.pickle", 'wb') as f:
    pickle.dump(neur_ind['gene_dict'], f)
    f.close()

### Stratified by batch

In [23]:
neur_batch = get_pseudobulk(neur, stratifier="Batch")

In [122]:
np.savetxt( "../data/neur.dpt.batch.X.txt", neur_batch['X'])
np.savetxt("../data/neur.dpt.batch.Y.txt", neur_batch['Y'])
with open("../data/neur.dpt.batch.genedict.pickle", 'wb') as f:
    pickle.dump(neur_batch['gene_dict'], f)
    f.close()

In [24]:
np.savetxt( "../data/neur.dpt.hires.batch.X.txt", neur_batch['X'])
np.savetxt("../data/neur.dpt.hires.batch.Y.txt", neur_batch['Y'])
with open("../data/neur.dpt.hires.batch.genedict.pickle", 'wb') as f:
    pickle.dump(neur_batch['gene_dict'], f)
    f.close()

### Stratified by batch and individual

In [25]:
neur.obs['batchind'] = [neur.obs['individual'][i] + '--' + neur.obs['Batch'][i] for i in range(neur.X.shape[0])]

In [26]:
neur_batchind = get_pseudobulk(neur, stratifier="batchind")

In [20]:
np.savetxt( "../data/neur.dpt.batchind.X.txt", neur_batchind['X'])
np.savetxt("../data/neur.dpt.batchind.Y.txt", neur_batchind['Y'])
with open("../data/neur.dpt.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(neur_batchind['gene_dict'], f)
    f.close()
with open("../data/neur.dpt.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(neur_batchind['level_dict'], f)
    f.close()

In [27]:
np.savetxt( "../data/neur.dpt.hires.batchind.X.txt", neur_batchind['X'])
np.savetxt("../data/neur.dpt.hires.batchind.Y.txt", neur_batchind['Y'])
with open("../data/neur.dpt.hires.batchind.genedict.pickle", 'wb') as f:
    pickle.dump(neur_batchind['gene_dict'], f)
    f.close()
with open("../data/neur.dpt.hires.batchind.linedict.pickle", 'wb') as f:
    pickle.dump(neur_batchind['level_dict'], f)
    f.close()